In [1]:
import os
#abspath = os.path.abspath(__file__)
#dname = os.path.dirname(abspath)
#os.chdir(dname)

import torch
import torchvision
import sys
sys.path.append('../')
#sys.path.append('../file_helper.py')
from config import *
from tqdm import tqdm
from model import *
from file_helper import *

# DataLoader
import torchvision.transforms as T
from torchvision import transforms
from torchvision.transforms import ToTensor
from PIL import Image
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

#EVAL_NAME= "faster_rcnn_ep60_test_1k_0107"
EVAL_NAME= "___Test"
writer = SummaryWriter(EVAL_NAME)

from train import *
from PE_helper import *

transform = T.Compose([
        transforms.Resize((512, 1024)),
        T.ToTensor(),  # convert PIL image to PyTorch tensor
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalize image
])

c:\Users\User\anaconda3\envs\Layout\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\User\anaconda3\envs\Layout\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


e:\Projects\Layout\Horizon_and_SAM\Horizon


In [2]:
def uv_to_box(u,v):
    return np.array([u[0],v[0] , u[1],v[2] , u[1],v[3],u[0],v[1]]).reshape(-1,2)    

def split_cross_boundary_bbox (bbox , w = 2048 ):
    results= []
    bbox = bbox.flatten()
    max_gt = np.max(bbox.flatten())    
    if(max_gt>w):
        results.append(np.array([bbox[0]  ,bbox[1]  , w , bbox[1] , w , bbox[5] , bbox[0] , bbox[5]]))  
        results.append(np.array([0  ,bbox[1]  , bbox[2]%w , bbox[1] , bbox[2]%w , bbox[5] , 0,bbox[5]]))  
    else:
        results = [bbox]
    pass
    #results = np.array(results)
    return results
def xyxy_to_bbox_polygon(xyxy):
    return np.array([xyxy[0] ,xyxy[1] , xyxy[2],xyxy[1] , xyxy[2],xyxy[3] , xyxy[0] , xyxy[3]  ]).reshape(-1,2)

In [6]:
#dataset_test = ZillowDataset(transforms=transform , anno_path= '../anno/test_visiable_10.json' )
dataset_test = ZillowDataset(transforms=transform , anno_path= '../anno/test_visiable_all.json' )
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=3, shuffle=False, num_workers=0  , collate_fn = collate_fn )

pt_path = os.path.join(os.getcwd() , "checkpoints","ep60.pth")
model_2cls.load_state_dict(torch.load(pt_path))    
model_2cls = model_2cls.to('cuda')

prediction_output_folder = create_folder(os.path.join(os.getcwd() , "eval"))
def do_eval(model , data_loader , run_name , prediction_output_folder , show_plt = False):    

    output_file_path = os.path.join(prediction_output_folder , f"eval_{run_name}.json")
    model.eval()
    # create new eval helper object
    writer = SummaryWriter(run_name)
    eval_helper = PR_Eval_Helper(get_iou_fn = get_iou_matrix_pixel_level , writer= writer)

    # record file to write
    prediction_outputs = {"data":[]}

    for data in tqdm(data_loader):
        img_b , anno_b = data
        anno_b_list = anno_to_list(anno_b)        
        output = model(img_b , anno_b_list)

        for img , out , gt in zip(img_b , output, anno_b):
            predict_out= {"image": gt['image_file']  , "pred": [] , "iou":[] }
            gt_seg_mask =[]
            out_bbox_mask=[]
            '''
            #==============================
            #           Debug
            #==============================
            # Pred
            debug_img =  debug_draw_bbox(img, out['boxes'].detach().cpu().numpy() , normalize=False)
            plt.imshow(debug_img)
            plt.show()
            
            # GT
            debug_img =  debug_draw_bbox(img, gt['boxes'].detach().cpu().numpy() , normalize=False , color=(0,0,255))
            plt.imshow(debug_img)
            plt.show()
            '''

            h,w = img.shape[-2:]
            gt_u , gt_v = torch.as_tensor(gt['u']) , torch.as_tensor(gt['v'])
            
            #============================
            #     Pred Bbox iou                    
            #============================            
            for box in out['boxes']:   
                predict_out['pred'].append(box.detach().cpu().numpy().tolist())
                door_parts = split_cross_boundary_bbox(xyxy_to_bbox_polygon(box.detach().cpu().numpy()) , w=w)            
                
                _mask_canvas = np.zeros((h,w))
                for debug_door_part in door_parts:            
                    debug_door_part = debug_door_part.reshape(-1,2)

                    mask = cv2.fillPoly(_mask_canvas , [np.int32(debug_door_part )]  , (255,255,255) )
                    '''
                    '''
                    if show_plt:
                        plt.imshow(mask)
                        plt.show()
                    out_bbox_mask.append(mask)
                pass

            #============================
            #     GT Seg
            #============================
            gt_seg_u =  gt_u
            gt_seg_vt = gt_v.flatten()[::2].view(-1,2)
            gt_seg_vb = gt_v.flatten()[1::2].view(-1,2)
            for u,vt,vb in zip(gt_seg_u , gt_seg_vt , gt_seg_vb):
                polys , mask = to_distorted_box([u] , [vt] , [vb] , h=h , w = w , show_plt=show_plt , seg_count=5)            
                mask= np.max(mask,axis=2)
                gt_seg_mask.append(mask)
            
            
            iou_list =  eval_helper.eval_batch_pr(out_bbox_mask,gt_seg_mask ,None )
            #  Append IoU
            predict_out['iou'] = iou_list
            if show_plt:
                print("iou_list " , iou_list  )
            prediction_outputs['data'].append(predict_out)

    mIou = eval_helper.get_all_pr( show_plt = show_plt)
    print("auc@0.5" , eval_helper.final_result_dict[1]['ap'])

    if show_plt:
        print("predict_out" , prediction_outputs)

    prediction_outputs["mIou"]   =  mIou.astype(float)
    prediction_outputs["ap@0.05"]   =  eval_helper.final_result_dict[0]['ap']
    prediction_outputs["ap@0.5"]    =  eval_helper.final_result_dict[1]['ap']
    prediction_outputs["ap@0.75"]   =  eval_helper.final_result_dict[2]['ap']
    prediction_outputs["recall@0.05"]   =  eval_helper.final_result_dict[0]['recall_rate']
    prediction_outputs["recall@0.5"]    =  eval_helper.final_result_dict[1]['recall_rate']
    prediction_outputs["recall@0.75"]   =  eval_helper.final_result_dict[2]['recall_rate']
    prediction_outputs["precision@0.05"]    =  eval_helper.final_result_dict[0]['precision_rate']
    prediction_outputs["precision@0.5"]     =  eval_helper.final_result_dict[1]['precision_rate']
    prediction_outputs["precision@0.75"]    =  eval_helper.final_result_dict[2]['precision_rate']

    json_object = json.dumps(prediction_outputs) 
    with open( output_file_path , "w" ) as f :
        f.write(json_object)
        pass

    return eval_helper.final_result_dict[1]['ap']

#model_folder = os.path.join(os.getcwd() , "checkpoints" , "ToDo_eval_folder")   # ToDo: set folder
model_folder = "E:/OneDrive - NTHU/Layout/Faster_Rcnn/Lab_0126_for_eval/train_10k_final-4-start_ep60"
model_files = get_files_with_prefix(model_folder , "")

best_ap = 0
best_mode_path =""
best_model_list =  []
for model_path in model_files:
    model_2cls.load_state_dict(torch.load(model_path))    
    model_2cls = model_2cls.to('cuda')

    ap_05 = do_eval(model_2cls , data_loader_test , os.path.basename(model_path) , prediction_output_folder , False)

    best_model_list.append({"model_path": model_path , "ap@0.5" : ap_05})
    if best_ap < ap_05:
        best_ap = ap_05
        best_mode_path = model_path
    
print("best model" , best_mode_path)
print("best ap " , best_ap)

json_object = json.dumps(best_model_list) 
output_file_path = os.path.join(os.getcwd() , "eval" , "model_performance_list.json")
with open( output_file_path , "w" ) as f :
    f.write(json_object)
    pass




anno loaded  6249


100%|██████████| 2083/2083 [24:25<00:00,  1.42it/s]


ap_0.05 0.6694781468764994
ap_0.5 0.5723927531485592
ap_0.75 0.37111789615153556
all_iou 7944.3400623917905
mIOU 0.6137943338014209
all_iou 7944.3400623917905
mIOU 0.6137943338014209
all_iou 7944.3400623917905
mIOU 0.6137943338014209
auc@0.5 0.5723927531485592


100%|██████████| 2083/2083 [25:51<00:00,  1.34it/s]


ap_0.05 0.6635194470900144
ap_0.5 0.5486082689816854
ap_0.75 0.33132511235336926
all_iou 8297.81202281937
mIOU 0.6411042279857351
all_iou 8297.81202281937
mIOU 0.6411042279857351
all_iou 8297.81202281937
mIOU 0.6411042279857351
auc@0.5 0.5486082689816854


...

ap_0.05 0.6746896528819316
ap_0.5 0.560931790201589
ap_0.75 0.3429426511284639
all_iou 8379.156929631656
mIOU 0.647389085191351
all_iou 8379.156929631656
mIOU 0.647389085191351
all_iou 8379.156929631656
mIOU 0.647389085191351
auc@0.5 0.560931790201589


100%|██████████| 2083/2083 [23:05<00:00,  1.50it/s]


ap_0.05 0.6716271007614376
ap_0.5 0.5802932885187807
ap_0.75 0.36909419161554
all_iou 8019.511257224644
mIOU 0.6196021986575481
all_iou 8019.511257224644
mIOU 0.6196021986575481
all_iou 8019.511257224644
mIOU 0.6196021986575481
auc@0.5 0.5802932885187807


100%|██████████| 2083/2083 [23:01<00:00,  1.51it/s]


ap_0.05 0.6786685395368015
ap_0.5 0.5729996529228019
ap_0.75 0.36878992987750736
all_iou 8006.677874104953
mIOU 0.6186106678594571
all_iou 8006.677874104953
mIOU 0.6186106678594571
all_iou 8006.677874104953
mIOU 0.6186106678594571
auc@0.5 0.5729996529228019
best model E:/OneDrive - NTHU/Layout/Faster_Rcnn/Lab_0126_for_eval/train_10k_final-4-start_ep60\ep80.pth
best ap  0.5835080297974726
